In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.drawing.image import Image
import os

# Load dataset
df = pd.read_csv('/content/trading_data.csv')

# Data preparation
df['Date'] = pd.to_datetime(df['Date'])
df['Direction'] = df['Direction'].map({'Buy': 1, 'Sell': -1})
df['Profit_Loss'] = (df['Exit_Price'] - df['Entry_Price']) * df['Quantity'] * df['Direction']

# Function to create charts
def create_charts(data):
    charts = {}

    # 1. Profit/Loss Distribution
    plt.figure(figsize=(8, 6))
    sns.histplot(data['Profit_Loss'], kde=True, bins=30, color='blue')
    plt.title('Distribution of Profit/Loss')
    plt.xlabel('Profit/Loss')
    plt.ylabel('Frequency')
    chart_path1 = '/content/profit_loss_distribution.png'
    plt.savefig(chart_path1)
    plt.close()
    charts['Profit/Loss Distribution'] = chart_path1

    # 2. SL% and Trgt% Comparison
    plt.figure(figsize=(8, 6))
    sns.boxplot(data=data[['SL%', 'Trgt%']])
    plt.title('Comparison of SL% and Trgt%')
    chart_path2 = '/content/sl_target_comparison.png'
    plt.savefig(chart_path2)
    plt.close()
    charts['SL and Target Comparison'] = chart_path2

    # 3. Correlation Heatmap
    plt.figure(figsize=(8, 6))
    correlations = data[['Entry_PCR', 'Exit_PCR', 'SL%', 'Trgt%', 'Profit_Loss']].corr()
    sns.heatmap(correlations, annot=True, cmap='coolwarm')
    plt.title('Correlations Heatmap')
    chart_path3 = '/content/correlations_heatmap.png'
    plt.savefig(chart_path3)
    plt.close()
    charts['Correlations Heatmap'] = chart_path3

    # 4. Average Metrics by Index
    metrics_by_index = data.groupby('Index')[['SL%', 'Trgt%', 'Profit_Loss']].mean().reset_index()
    plt.figure(figsize=(8, 6))
    metrics_by_index.plot(x='Index', kind='bar', figsize=(10, 6))
    plt.title('Average Metrics by Index')
    plt.xlabel('Index')
    plt.ylabel('Average Values')
    plt.xticks(rotation=45)
    chart_path4 = '/content/average_metrics_by_index.png'
    plt.savefig(chart_path4)
    plt.close()
    charts['Average Metrics by Index'] = chart_path4

    return charts

# Generate charts
charts = create_charts(df)

# Save results to Excel
output_file = "/content/trading_analysis_with_charts(assignment1_abcd).xlsx"

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # Raw Data
    df.to_excel(writer, sheet_name='Raw Data', index=False)

    # Strategy Summary
    strategy_summary = {
        "Average Entry PCR": df['Entry_PCR'].mean(),
        "Average Exit PCR": df['Exit_PCR'].mean(),
        "Average SL%": df['SL%'].mean(),
        "Average Trgt%": df['Trgt%'].mean(),
        "Total Trades": len(df),
        "Total Profit/Loss": df['Profit_Loss'].sum()
    }
    summary_df = pd.DataFrame(strategy_summary.items(), columns=['Metric', 'Value'])
    summary_df.to_excel(writer, sheet_name='Strategy Summary', index=False)

    # Correlations
    correlations = df[['Entry_PCR', 'Exit_PCR', 'SL%', 'Trgt%', 'Profit_Loss']].corr()
    correlations.to_excel(writer, sheet_name='Correlations')

    # Add Charts Sheet
    workbook = writer.book
    charts_sheet = workbook.create_sheet(title="Charts")

    # Add chart images to Charts sheet
    for idx, (chart_title, chart_path) in enumerate(charts.items(), start=1):
        img = Image(chart_path)
        img.anchor = f'A{(idx - 1) * 20 + 1}'  # Position each chart sequentially
        charts_sheet.add_image(img)

# Confirm Excel file creation
output_file if os.path.exists(output_file) else "Error creating file"


'/content/trading_analysis_with_charts(assignment1_abcd).xlsx'

<Figure size 800x600 with 0 Axes>